# Context-Free Grammars

In [1]:
import nltk

In [2]:
# NLTK makes it easy to generate a CFG by providing a helper function.

groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

In [3]:
# we created a CFG, which we can then feed into a parser object. 

parser = nltk.ChartParser(groucho_grammar)

Now that we have a parser, we can use it to parse sentences by passing in a tokenized sentence to the parser's .parse() method. This will return 0 or more Parse Trees for that sentence. If the sentence cannot be parsed, it will return no trees. If there is more than one grammatically valid way (according to the rules defined in our CFG) to parse the sentence, then it will return them all.

In [4]:
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
for tree in parser.parse(sent):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


## Parsing A Sentence

Create a sample grammar. The grammar can be broken down into two sections:

Rules defining valid compositions for Sentences (S), Prepositional Phrases (PP), Noun Phrases (NP), and Verb Phrases (VP). These are incomplete, and we'll need to modify them to get the sentences to parse correctly

Mappings that define the POS tag for each word we'll be working with, as seen from the line Det -> and all the lines after it. These will be provided for you at each step, so that you only have to worry about the rules

- Step 2 defines the sentences we'll be working with as strings
- Step 3 creates a tokenized version of each sentence
- Step 4 creates a new parser with our grammar CFG
- Step 5 tries to parse the tokenized version of the sentence

In [6]:
# Step 1
grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 
VP -> V NP | VP PP
Det -> 'the'
Adj -> '100m'
N -> 'usain_bolt' | 'record' | 
V -> 'broke'
P -> 
""")

In [7]:
# Step 2
from nltk import word_tokenize
sent = 'usain_bolt broke the 100m record'

In [8]:
# Step 3
tokenized_sent = word_tokenize(sent)

In [9]:
# Step 4
parser = nltk.ChartParser(grammar)

In [10]:
# Step 5
for tree in parser.parse(tokenized_sent):
    print(tree)

The cell above didn't print anything, which means that our parser was unable to parse tokenized_sent correctly. Although it recognizes all the words (it would throw an error if it came across an unrecognized word), it doesn't contain the rules necessary to correctly parse this sentence.

- Noun Phrases (NP) need to be able to consist of Det followed by a Noun Phrase NP.
- Noun Phrases need to be able to consist of Adj followed by an N.

Let's try reloading our grammar, and reparsing our sentence.

- Modify and recreate our CFG using our updated rules
- Recreate our parser with the updated rules
- Try to parse tokenized_sent and see if we get any output

In [13]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N PP | N | Det NP | Adj NP
VP -> V NP | VP PP
Det -> 'the'
Adj -> '100m'
N -> 'usain_bolt' | 'record' | 
V -> 'broke'
P -> 
""")

In [14]:
parser = nltk.ChartParser(grammar)

In [15]:
for tree in parser.parse(tokenized_sent):
    print(tree)

(S
  (NP (N usain_bolt))
  (VP (V broke) (NP (Det the) (NP (Adj 100m) (NP (N record))))))
(S
  (NP (N usain_bolt))
  (VP
    (V broke)
    (NP (Det the) (N ) (PP (P ) (NP (Adj 100m) (NP (N record)))))))
(S
  (NP (N usain_bolt))
  (VP
    (VP (V broke) (NP (N )))
    (PP (P ) (NP (Det the) (NP (Adj 100m) (NP (N record)))))))
(S
  (NP (N usain_bolt))
  (VP
    (VP (V broke) (NP (N )))
    (PP
      (P )
      (NP (Det the) (N ) (PP (P ) (NP (Adj 100m) (NP (N record))))))))
(S
  (NP (N usain_bolt))
  (VP
    (VP (VP (V broke) (NP (N ))) (PP (P ) (NP (N ))))
    (PP (P ) (NP (Det the) (NP (Adj 100m) (NP (N record)))))))
(S
  (NP (N usain_bolt))
  (VP
    (VP (VP (V broke) (NP (N ))) (PP (P ) (NP (N ))))
    (PP
      (P )
      (NP (Det the) (N ) (PP (P ) (NP (Adj 100m) (NP (N record))))))))
(S
  (NP (N usain_bolt))
  (VP
    (VP (V broke) (NP (Det the) (NP (N ))))
    (PP (P ) (NP (Adj 100m) (NP (N record))))))
(S
  (NP (N usain_bolt))
  (VP
    (VP (V broke) (NP (Det the) (N ) (PP (P ) (

It worked! But why are there so many trees? Because as our rules stand, there are multiple valid interpretations of the sentence. This is because we have some empty things like PP -> P NP that don't currently have rules, so they're triggering as false positives.

So far, we've been manually labeling our words to construct our CFGs. However, this strategy obviously doesn't scale well to the real world -- it would take way too long to label every possible word with a part of speech, and generate every rule possible for a CFG. 

Luckily, the linguists and programmers have provided a quick and easy way to have this done for us, thanks to the Penn Tree Bank! The Penn Tree Bank is "a large annotated corpus of English", according to the researchers at Penn that created it. This means that it already has a ton of well-labeled examples for POS tagging. 

For us, this means that we can easily use NLTK to generate POS tags for our corpora, without having to worry about hand labeling or generating parse trees.

Generating POS tags is very simple with NLTK -- all we need to do is pass in a tokenized version of our corpus and NLTK will return a list of tuples containing the token and it's part of speech.

Note that the abbreviations NLTK uses for their POS tags come from the Penn Tree Bank. To understand what these tags stand for, take a look at this reference: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [16]:
# This is all it takes to generate POS tags using NLTK.

nltk.pos_tag(tokenized_sent)

[('usain_bolt', 'JJ'),
 ('broke', 'VBD'),
 ('the', 'DT'),
 ('100m', 'CD'),
 ('record', 'NN')]